## DOCUMENTATION  
this is just the initial documenatation whcih will be improved once we integrate in Github

In [ ]:
# Date        Initials    Description
# 10FEB2023   PN          - Initial Library, USGS Load for 2023, 48 Philippines stations, 50/10 data extract, RasberryShape Data Extract loop


#Future add ASAP (as of 5 April 2024)
# MMF 3 year data extract loop, Initial Arima, K-Mean, and LSTM model mode, Log/Lat extraction for each station, and Closest 5 stations to eartquake loop
# PN STA and LTA features, Auto Trigger function

## Initial PIP installation


In [ ]:
# ! pip install obspy # enable line when using temporary notebook/instance
# ! mkdir /content/rs-usgs-test
# ! cd /content/rs-usgs-test && gdown https://drive.google.com/uc?id=1K4ivaWZBZi_26JmFwqOH1GP2q8jO15Rv

In [ ]:
# Instance information for any diagnostics use

! python --version
! nproc

Python 3.10.12
2


## Library Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt

import obspy

import cartopy.crs as ccrs
import cartopy.feature as cfeature

from obspy.clients.fdsn import Client
from obspy.core import UTCDateTime, Stream, Trace
from obspy.imaging.maps import plot_cartopy as plt_crt

import os
import threading
import time

from concurrent.futures import ThreadPoolExecutor as ccf_tpe
from concurrent.futures import ProcessPoolExecutor as ccf_ppe

## Loading USGS Dataset with every earthquake

Please make sure to add the "assets" folder in the same folder as ML004 notebook

In [ ]:
#usgs_data = '/content/rs-usgs-test/DS001_USGS_dataset 1950-2023.csv'
usgs_data = 'assets/DS001_USGS_dataset 1950-2023.csv'

usgs_df = pd.read_csv(usgs_data)                # loads csv to dataframe
usgs_df.time = pd.to_datetime(usgs_df.time)     # convert object type to datetime64

year_lim = 2023

usgs_df_lim = usgs_df[usgs_df.time.dt.year >= year_lim]
usgs_df_lim.reset_index(inplace=True)

In [ ]:
filtered = usgs_df_lim[(usgs_df_lim['mag'] >= 5) & (usgs_df_lim['depth'] <= 10)].reset_index()
filtered.columns

## Manually finding the Rasberry Shakestations in the Philippines

In [ ]:
stations_luzon = ['R1EFC',
                'S050E',
                'R061D',
                'RE7F4',
                'R3B2D',
                'R6423',
                'R5160',
                'R67F1',
                'RD61C',
                'R5595',
                'R1CB3',
                'R3F4D',
                'R1CB3',
                'S6A6F',
                'RA66D',
                'R2960',
                'R1201',
                'RC58C',
                'R3A55',
                'R3282',
                'RE63C',
                'R4F2C',
                'RD4DD',
                'R191E',
                'R5744',
                'RC8A0',
                'R9912',
                'RBBDE',
                'R1E7D',
                'R91E4',
                'S690A',
                'R98E0',
                'R8095'] # last station for Luzon is in CamSur

stations_visayas = ['R257C',
                    'R9829',
                    'R9CAE',
                    'R402F',
                    'R23C6',
                    'REA68',
                    'R4C3D',
                    'RF17C'] # last station for Visayas is in Naga



stations_mindanao = ['REBEA',
                     'R2B9C',
                     'RD5D8',
                     'RF8C0',
                     'R4893',
                     'R1382',
                     'RBD68'] # last station in Mindanao is in Tulunan


stations = stations_luzon + stations_visayas + stations_mindanao

## Taking 50 minutes before and 10 minutes after

In [ ]:
## taking 50 minutes before and 10 minutes after
t_start : int = 3000
t_end   : int = 600

event : float = UTCDateTime(usgs_df.time[0].year,
                    usgs_df.time[0].month,
                    usgs_df.time[0].day,
                    usgs_df.time[0].hour,
                    usgs_df.time[0].minute,
                    usgs_df.time[0].second)

start : float = event - t_start
end : float   = event + t_end

In [ ]:
# get station metadata and put into df
rs = Client('RASPISHAKE')
net = 'AM'
channels = ['EHZ', 'HDF','ENE','ENN', 'ENZ']
stream = Stream()

## Getting Rasberryshake Data from the 48 stations for every earthquake in USGS data

In [ ]:
def ToFile(data, stations, filename, format):

    t_start : int = 3000
    t_end   : int = 600

    rs = Client('RASPISHAKE')
    net = 'AM'
    channels = ['EHZ', 'HDF','ENE','ENN', 'ENZ']
    stream = Stream()

    NumEvents = data.index

    for index in NumEvents:
        event = UTCDateTime(data.time[index].year,
                        data.time[index].month,
                        data.time[index].day,
                        data.time[index].hour,
                        data.time[index].minute,
                        data.time[index].second)

        start : float = event - t_start
        end : float   = event + t_end

        for stn in stations:
            for chan in channels:
                try:
                    trace = rs.get_waveforms(net, stn, '00', chan, start, end)
                    stream += trace

                except Exception: #HTTPError i.e. channel or station does not exist
                    pass

        stream.write(f'{filename}_{index}.pkl', format = format)



In [ ]:
# ToFile(data = filtered, stations = stations, filename = 'rsn_data', format = 'PICKLE')